<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-05 12:06:29
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.39 C
-------------------
Today PnL: -81.44 K (-0.58%)
Current PnL: -30.77 L (-19.62%)
CY Booked + Current PnL: -16.03 L (-10.22%)
-------------------
Total profit:  1.26 L
Total loss:  -32.03 L
-------------------
Total Booked + Current PnL: 10.95 L (8.44%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.63%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 96.15 L (69.31%)
Deployed:  1.30 C
Current:  1.39 C
CAGR/XIRR %: 3.46%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.62,6.28,12.18,19.23,19478.0,9454.0,159914.0,484.83,24.27,55.0,L-LC,12.37,182.0,0.49,1.15,28.90,XY25,ATH,MINING
50,MASFIN,1.19,-3.55,26.04,21.57,24608.0,-3480.0,94500.0,397.04,-16.89,51.0,H-SC,3.50,168.0,-0.14,0.68,37.90,XR,ATH,FINANCE
77,TTKPRESTIG,0.17,-27.78,38.59,0.09,28087.0,-27993.0,72784.0,770.00,68.84,34.0,M-SC,7.91,253.0,-1.00,0.52,0.17,OX40N,NTT,DURABLES
57,RECLTD,0.37,3.68,16.45,20.74,34652.0,7480.0,210650.0,446.00,35.83,61.0,M-MC,3.98,189.0,0.22,1.52,14.69,XY25,NTT,FINANCE
51,MEDANTA,-2.50,-8.60,35.74,24.08,41472.0,-10912.0,116038.0,1486.00,-15.92,46.0,X-SC,11.16,91.0,-0.26,0.84,6.28,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,WHIRLPOOL,2.97,-34.36,177.17,81.92,133494.0,-39450.0,75348.0,2270.0,-61.52,45.0,M-SC,32.98,218.0,-0.30,0.54,5.84,XR,NTT,DURABLES
14,BSOFT,2.75,-17.97,83.82,50.79,96707.0,-25278.0,115375.0,831.7,5.14,62.0,H-SC,6.54,171.0,-0.26,0.83,34.33,XR,ATH,IT
63,SFL,2.03,-39.64,109.78,26.63,173763.0,-103935.0,158283.0,1287.0,19.88,66.0,M-SC,16.26,241.0,-0.60,1.14,21.50,XY24,NTT,MISC
22,DEN,1.46,-44.53,156.15,42.07,73153.0,-37616.0,46848.0,75.0,65.80,48.0,M-SC,13.83,236.0,-0.51,0.34,7.61,AR,NTT,ENTERTAINMENT
66,SIS,1.33,-19.62,50.86,21.26,45393.0,-21782.0,89250.0,528.0,2111.40,66.0,H-SC,4.66,163.0,-0.48,0.64,20.59,OX40N,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJOOENG,-4.14,-37.63,123.84,39.61,79171.0,-38570.0,63930.0,143.1,-57.76,47.0,H-SC,29.45,136.0,-0.49,0.46,7.10,AR,ATH,MISC
84,WIPRO,-3.85,-4.29,79.99,72.27,129721.0,-7277.0,162171.0,420.0,-17.88,38.0,M-LC,2.55,101.0,-0.06,1.17,4.24,XR,NTT,IT
19,COFFEEDAY,-3.15,-48.58,163.07,35.27,95212.0,-55162.0,58387.0,80.0,-59.76,38.0,L-SC,12.87,268.0,-0.58,0.42,42.90,XR,NTT,HOTELS
64,SHALBY,-2.94,-34.45,90.84,25.10,107869.0,-62397.0,118746.0,327.0,772.41,41.0,M-SC,15.02,248.0,-0.58,0.86,4.01,XY24,NTT,HEALTHCARE
76,TRIDENT,-2.90,-23.91,68.90,28.51,48366.0,-22058.0,70197.0,48.0,-10.08,62.0,M-SC,9.91,226.0,-0.46,0.51,20.12,XR,NTT,TEXTILES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,0.01,-5.64,120.34,107.91,167652.0,-8329.0,139315.0,1641.55,-23.39,50.0,M-SC,11.51,220.0,-0.05,1.00,4.81,OX40N,ATH,CABLES
66,SIS,1.33,-19.62,50.86,21.26,45393.0,-21782.0,89250.0,528.00,2111.40,66.0,H-SC,4.66,163.0,-0.48,0.64,20.59,OX40N,NTT,MISC
72,TATAELXSI,-1.85,-22.06,69.40,32.03,71310.0,-29083.0,102752.0,9161.00,-15.70,49.0,H-SC,6.38,157.0,-0.41,0.74,15.09,OX40N,NTT,IT
48,KANSAINER,-1.60,-27.07,55.59,13.47,109328.0,-72999.0,196668.0,340.00,-70.06,37.0,H-SC,5.58,159.0,-0.67,1.42,0.00,XY24,NTT,PAINTS
77,TTKPRESTIG,0.17,-27.78,38.59,0.09,28087.0,-27993.0,72784.0,770.00,68.84,34.0,M-SC,7.91,253.0,-1.00,0.52,0.17,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,0.37,3.68,16.45,20.74,34652.0,7480.0,210650.0,446.00,35.83,61.0,M-MC,3.98,189.0,0.22,1.52,14.69,XY25,NTT,FINANCE
18,COALINDIA,-0.62,6.28,12.18,19.23,19478.0,9454.0,159914.0,484.83,24.27,55.0,L-LC,12.37,182.0,0.49,1.15,28.90,XY25,ATH,MINING
1,ABB,1.17,6.83,36.26,45.57,101339.0,17861.0,279480.0,7934.00,-31.40,73.0,H-MC,5.14,121.0,0.18,2.02,24.11,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,1.17,6.83,36.26,45.57,101339.0,17861.0,279480.0,7934.00,-31.40,73.0,H-MC,5.14,121.0,0.18,2.02,24.11,AR,NTT,ELECTRICAL
39,INDIAMART,0.01,-2.22,111.42,106.72,134368.0,-2740.0,120596.0,4810.62,-51.58,57.0,H-SC,8.49,138.0,-0.02,0.87,20.64,AR,ATH,MISC
26,FINCABLES,0.01,-5.64,120.34,107.91,167652.0,-8329.0,139315.0,1641.55,-23.39,50.0,M-SC,11.51,220.0,-0.05,1.00,4.81,OX40N,ATH,CABLES
84,WIPRO,-3.85,-4.29,79.99,72.27,129721.0,-7277.0,162171.0,420.00,-17.88,38.0,M-LC,2.55,101.0,-0.06,1.17,4.24,XR,NTT,IT
85,ZYDUSLIFE,0.08,-4.46,41.96,35.63,84015.0,-9348.0,200226.0,1286.17,-18.07,54.0,H-MC,6.06,119.0,-0.11,1.44,11.47,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-0.79,-44.94,145.55,35.20,103121.0,-57821.0,70849.0,2096.00,-7.15,25.0,X-SC,20.90,93.0,-0.56,0.51,0.68,X40,NTT,FOOTWEAR
44,ITC,-1.08,-23.91,45.60,10.78,83382.0,-57463.0,182855.0,452.00,-54.64,27.0,X-LC,18.81,1.0,-0.69,1.32,0.72,X40,NTT,FMCG
81,VBL,-1.35,-11.34,53.03,35.67,148494.0,-35824.0,280018.0,671.64,-21.86,27.0,X-LC,0.29,5.0,-0.24,2.02,0.94,X40N,ATH,FMCG
6,ASIANPAINT,-1.41,-15.48,43.10,20.95,91710.0,-38984.0,212784.0,3460.25,-16.37,28.0,X-LC,9.52,36.0,-0.43,1.53,14.63,X40,ATH,PAINTS
13,BERGEPAINT,-1.17,-15.47,43.31,21.14,83230.0,-35167.0,192172.0,680.00,-21.44,32.0,X-MC,8.44,75.0,-0.42,1.39,2.51,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VBL,-1.35,-11.34,53.03,35.67,148494.0,-35824.0,280018.0,671.64,-21.86,27.0,X-LC,0.29,5.0,-0.24,2.02,0.94,X40N,ATH,FMCG
73,TCS,-0.10,-17.62,43.91,18.55,136817.0,-66653.0,311584.0,4311.59,-29.13,38.0,X-LC,0.48,3.0,-0.49,2.25,6.00,X40,ATH,IT
5,ANGELONE,-1.52,8.35,14.63,24.20,48385.0,25475.0,330725.0,3033.00,54.39,56.0,X-SC,1.43,99.0,0.53,2.38,38.51,X40N,NTT,FINANCE
34,HINDUNILVR,0.27,-7.52,22.91,13.67,54466.0,-19323.0,237740.0,2922.00,-14.44,52.0,X-LC,2.38,9.0,-0.35,1.71,12.23,XY25,NTT,FMCG
21,DABUR,0.87,-1.16,45.39,43.70,111293.0,-2876.0,245192.0,735.00,-9.24,46.0,X-MC,2.40,73.0,-0.03,1.77,13.72,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-0.79,-44.94,145.55,35.20,103121.0,-57821.0,70849.0,2096.00,-7.15,25.0,X-SC,20.90,93.0,-0.56,0.51,0.68,X40,NTT,FOOTWEAR
44,ITC,-1.08,-23.91,45.60,10.78,83382.0,-57463.0,182855.0,452.00,-54.64,27.0,X-LC,18.81,1.0,-0.69,1.32,0.72,X40,NTT,FMCG
81,VBL,-1.35,-11.34,53.03,35.67,148494.0,-35824.0,280018.0,671.64,-21.86,27.0,X-LC,0.29,5.0,-0.24,2.02,0.94,X40N,ATH,FMCG
2,ABBOTINDIA,-0.20,-9.27,30.13,18.07,48892.0,-16585.0,162270.0,35195.00,-22.28,41.0,X-MC,9.33,64.0,-0.34,1.17,1.54,X40,ATH,PHARMA
53,PGHH,0.03,-11.44,49.07,32.01,92785.0,-24432.0,189088.0,17616.87,-37.74,40.0,X-MC,8.58,60.0,-0.26,1.36,2.20,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,0.82,-26.29,93.79,42.84,44939.0,-17091.0,47915.0,424.00,-51.52,62.0,X-SC,21.15,83.0,-0.38,0.35,12.63,XY24,NTT,MISC
58,RELAXO,-1.48,-52.02,217.45,52.31,151443.0,-75515.0,69645.0,1176.00,-48.89,41.0,X-SC,12.36,92.0,-0.50,0.50,3.56,X40N,NTT,FOOTWEAR
12,BATAINDIA,-0.79,-44.94,145.55,35.20,103121.0,-57821.0,70849.0,2096.00,-7.15,25.0,X-SC,20.90,93.0,-0.56,0.51,0.68,X40,NTT,FOOTWEAR
51,MEDANTA,-2.50,-8.60,35.74,24.08,41472.0,-10912.0,116038.0,1486.00,-15.92,46.0,X-SC,11.16,91.0,-0.26,0.84,6.28,XY24,NTT,HEALTHCARE
35,HONAUT,-0.70,-19.47,77.97,43.32,102265.0,-31712.0,131160.0,58357.33,-30.77,47.0,X-SC,9.15,90.0,-0.31,0.95,6.50,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-1.08,-23.91,45.60,10.78,83382.0,-57463.0,182855.0,452.00,-54.64,27.0,X-LC,18.81,1.0,-0.69,1.32,0.72,X40,NTT,FMCG
75,TMPV,-2.01,-24.60,63.09,22.98,107466.0,-55560.0,170338.0,600.00,-21.86,59.0,X-LC,4.41,2.0,-0.52,1.23,8.89,XY24,NTT,AUTO
73,TCS,-0.10,-17.62,43.91,18.55,136817.0,-66653.0,311584.0,4311.59,-29.13,38.0,X-LC,0.48,3.0,-0.49,2.25,6.00,X40,ATH,IT
81,VBL,-1.35,-11.34,53.03,35.67,148494.0,-35824.0,280018.0,671.64,-21.86,27.0,X-LC,0.29,5.0,-0.24,2.02,0.94,X40N,ATH,FMCG
42,INFY,-0.73,3.79,29.35,34.24,95311.0,11852.0,324740.0,1972.00,-19.24,33.0,X-LC,6.51,6.0,0.12,2.34,12.38,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-1.79,-33.20,98.93,32.89,51024.0,-25630.0,51576.0,1800.00,-736.93,52.0,L-MC,11.17,259.0,-0.50,0.37,42.04,XR,NTT,BANKS
67,SONACOMS,-1.35,-9.06,52.96,39.09,48717.0,-9170.0,91989.0,804.02,-28.90,69.0,M-SC,1.83,227.0,-0.19,0.66,30.13,AR,ATH,AUTO
14,BSOFT,2.75,-17.97,83.82,50.79,96707.0,-25278.0,115375.0,831.70,5.14,62.0,H-SC,6.54,171.0,-0.26,0.83,34.33,XR,ATH,IT
18,COALINDIA,-0.62,6.28,12.18,19.23,19478.0,9454.0,159914.0,484.83,24.27,55.0,L-LC,12.37,182.0,0.49,1.15,28.90,XY25,ATH,MINING
65,SIEMENS,0.53,-11.12,41.22,25.51,68178.0,-20695.0,165400.0,4671.50,1.56,69.0,H-LC,3.58,50.0,-0.30,1.19,21.89,AR,ATH,ELECTRICAL


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,1.17,6.83,36.26,45.57,101339.0,17861.0,279480.0,7934.00,-31.40,73.0,H-MC,5.14,121.0,0.18,2.02,24.11,AR,NTT,ELECTRICAL
5,ANGELONE,-1.52,8.35,14.63,24.20,48385.0,25475.0,330725.0,3033.00,54.39,56.0,X-SC,1.43,99.0,0.53,2.38,38.51,X40N,NTT,FINANCE
67,SONACOMS,-1.35,-9.06,52.96,39.09,48717.0,-9170.0,91989.0,804.02,-28.90,69.0,M-SC,1.83,227.0,-0.19,0.66,30.13,AR,ATH,AUTO
62,SATIN,-0.17,-12.34,73.21,51.83,177307.0,-34095.0,242189.0,274.00,-21.16,63.0,H-SC,3.23,148.0,-0.19,1.75,17.98,XY24,NTT,FINANCE
27,GILLETTE,-0.88,1.88,27.33,29.72,70696.0,4779.0,258675.0,10978.86,-16.31,64.0,X-SC,11.66,87.0,0.07,1.87,19.70,X40,ATH,FMCG


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.40
1,25,44.72
2,50,76.74


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.39
MC    30.87
LC    23.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.66
X40      23.35
X40N     15.07
AR        9.08
XY25      8.80
XR        8.76
OX40N     7.40
SR        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.06
H-SC    22.96
X-LC    20.25
M-SC    11.33
X-SC    10.34
H-MC     4.92
M-MC     1.52
H-LC     1.19
M-LC     1.17
L-LC     1.15
L-SC     0.76
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.90,-14.56,53.06
FINANCE,13.57,-11.03,54.72
IT,12.42,-24.07,86.06
MISC,7.22,-29.88,83.01
ELECTRICAL,6.19,-9.45,49.38
PAINTS,5.25,-26.97,45.83
INSURANCE,4.77,-2.43,37.49
PHARMA,3.98,-6.36,39.83
AUTO,2.96,-27.55,71.66


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3278972.0,21
AR,1311519.0,10
XR,1276147.0,12
X40,1257745.0,15
X40N,1032605.0,10
OX40N,764468.0,10
XY25,398621.0,6
SR,295175.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3610935.0,24
X-MC,1690946.0,16
M-SC,1487220.0,15
X-LC,1081101.0,12
X-SC,859751.0,9
H-MC,397132.0,3
L-SC,185114.0,2
M-LC,129721.0,1
H-LC,68178.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1234374.0      6
           AR         936117.0      5
M-SC       XY24       835863.0      6
H-SC       XR         818330.0      7
X-MC       X40        570781.0      7
X-LC       X40        513147.0      6
X-MC       XY24       479017.0      3
           X40N       416434.0      4
X-SC       X40N       376332.0      4
M-SC       OX40N      347627.0      5
H-SC       OX40N      326939.0      4
X-SC       XY24       309602.0      3
H-SC       SR         295175.0      2
X-LC       X40N       239839.0      2
X-MC       XY25       224714.0      2
H-MC       XY24       211778.0      1
X-LC       XY24       208338.0      2
H-MC       AR         185354.0      2
M-SC       XR         181860.0      2
X-SC       X40        173817.0      2
M-LC       XR         129721.0      1
M-SC       AR         121870.0      2
X-LC       XY25       119777.0      2
L-SC       XR          95212.0      1
           OX40N       89902.0      1
H-LC       AR          68178.0      1
L-MC       XR          51024.0      1
M-MC       XY25        34652.0      1
L-LC       XY25        19478.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
